In [29]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [30]:
final_dataset = pd.read_csv("./Processed_Data/final_dataframe.csv")
X = final_dataset["Comments"]
y = final_dataset["Type"]

In [31]:
final_dataset.head()

,Unnamed: 0,Comments,Type
0,0,You also have a great smile!,1
1,1,"Sylvia Plath ""roasted"" herself by putting her ...",0
2,2,"Be kinder to yourself, you're worth it.",1
3,3,You’re cute and you have good hair!,1
4,4,"If you could just move a little, over to my le...",0


In [32]:
final_dataset.groupby('Type').describe()
final_dataset["Type"].value_counts()

0    69397
1    36656
Name: Type, dtype: int64

In [33]:
df_roast = final_dataset[final_dataset["Type"] == 0]
df_toast = final_dataset[final_dataset["Type"] == 1]
df_roast_downsampled = df_roast.sample(df_toast.shape[0])
df_roast_downsampled

,Unnamed: 0,Comments,Type
74052,74052,"Look at that ""face.""",0
89935,89935,The definition of growing a beard to hide your...,0
51923,51923,What looks like a tongue sticking out is actua...,0
84471,84471,I say we go back to the lower resolution.,0
68492,68492,"Broken nails, broken face, no soul and you're ...",0
...,...,...,...
50035,50035,Why'd you cut your hair like a pornstar from 2...,0
28349,28349,Your makeup might fall off but those sharpied ...,0
36439,36439,Why is your face melting?,0
62080,62080,Head a straight up rectangle,0


In [34]:
dataset_balanced = pd.concat([df_roast_downsampled, df_toast])
dataset_balanced

,Unnamed: 0,Comments,Type
74052,74052,"Look at that ""face.""",0
89935,89935,The definition of growing a beard to hide your...,0
51923,51923,What looks like a tongue sticking out is actua...,0
84471,84471,I say we go back to the lower resolution.,0
68492,68492,"Broken nails, broken face, no soul and you're ...",0
...,...,...,...
106044,106044,That's a fucking awesome beard man!,1
106045,106045,Try to take back control a little bit at a time.,1
106046,106046,Bad ass style dude!,1
106051,106051,Dem brows 👌😍,1


In [35]:
dataset_balanced["Roast"] = dataset_balanced['Type'].apply(lambda x : 1 if x == 0 else 0)
dataset_balanced["Toast"] = dataset_balanced['Type'].apply(lambda x : 1 if x == 1 else 0)

In [36]:
X = dataset_balanced['Comments']
y = dataset_balanced['Roast']
X.head()

74052                                 Look at that "face."
89935    The definition of growing a beard to hide your...
51923    What looks like a tongue sticking out is actua...
84471            I say we go back to the lower resolution.
68492    Broken nails, broken face, no soul and you're ...
Name: Comments, dtype: object

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, stratify=dataset_balanced['Roast'])
X_train.head()

104008                    My god your eyes are breathtaking
88035     Your lack of trust in people makes total sense...
78621                Also, no one likes your stupid jacket.
4424                                Pick a fucking race, MJ
105189    Let whatever they throw at you roll right off ...
Name: Comments, dtype: object

In [42]:
preprocessor = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

FileNotFoundError: Op type not registered 'CaseFoldUTF8' in binary running on KETANSHARMA-PC. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.
 You may be trying to load on a different device from the computational device. Consider setting the `experimental_io_device` option in `tf.saved_model.LoadOptions` to the io_device such as '/job:localhost'.

In [40]:
model_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")

#BERT Layers
prepro_sent = preprocessor(model_input)
sent_embdings = encoder(prepro_sent)

#Neural Network Layers
layer1 = tf.keras.layers.Dropout(0.1, name="dropout")(sent_embdings['pooled_output'])
layer1 = tf.keras.layers.Dense(1, activation="sigmoid", name="dense")(layer1)

#Model
model = tf.keras.Model(input=[model_input], output=[layer1])

TypeError: 'str' object is not callable

In [41]:
model.summary()

NameError: name 'model' is not defined

In [ ]:
METRICS = [tf.keras.metrics.BinaryAccuracy(name="Accuracy"),
          tf.keras.metrics.Precision(name="precision"),
          tf.keras.metric.Recall(name="recall")]

model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=10)

In [ ]:
model.evaluate(X_test, y_test)


In [ ]:
y_predict = model.predict(X_test)
y_predict = y_predict.flatten()

In [ ]:
y_predict = np.where(y_predict > 0.5, 1, 0)
y_predict

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predict)
cm

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sbn
sbn.heatmap(cm, annot=True, fmt="d")
plt.xlabel("Predicted")
plt.ylabel("Truth")


In [ ]:
print(classification_report(y_test, y_predict))